In [44]:
import numpy as np
import pystan
from matustools.matusplotlib import saveStanFit

/usr/local/lib/python2.7/dist-packages/matplotlib-1.5.0-py2.7-linux-x86_64.egg/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [79]:
np.random.seed(5)

modelOneGroup='''
data{
    int<lower=0> N; //nr of subjects
    int<lower=0> T; //nr of trials
    real rt[N,T];
    int<lower=0,upper=1> ac[N,T];

}parameters{
    real<lower=0> sigma[N];
    vector[2] mu[N];
    vector[2] hm;
    vector<lower=0>[2] hsd;
    corr_matrix[2] r;  
}model{
    matrix[2,2] Theta;
    r~lkj_corr(1);
    Theta<-quad_form_diag(r,hsd);
    for (n in 1:N){
        mu[n]~multi_normal(hm,Theta);
    for (t in 1:T){
        if (rt[n,t]>0) rt[n,t]~lognormal(mu[n][1],sigma[n]);
        ac[n,t]~bernoulli_logit(mu[n][2]);
    }}   
}'''

modelMultiCondSameTheta='''
data{
    int<lower=0> N; //nr of subjects
    int<lower=0> T; //nr of trials
    int<lower=0> C; //nr of conditions
    int<lower=0> maxT[C]; // max nr of trials within each condition
    real rt[N,C,T];
    int<lower=-1,upper=1> ac[N,C,T];

}parameters{
    real<lower=0> sigma[N];
    vector<lower=-10,upper=10>[2] mu[N,C];
    vector<lower=-10,upper=10>[2] hm[C];
    vector<lower=0>[2] hsd;
    corr_matrix[2] r;  
}model{
    matrix[2,2] Theta;
    r~lkj_corr(1);
    Theta<-quad_form_diag(r,hsd);
    for (n in 1:N){
    for (c in 1:C){
        mu[n,c]~multi_normal(hm[c],Theta);
    for (t in 1:maxT[c]){
        if (rt[n,c,t]>0) rt[n,c,t]~lognormal(mu[n,c][1],sigma[n]);
        if (rt[n,c,t]<0) increment_log_prob(log1m(lognormal_cdf(-rt[n,c,t],mu[n,c][1],sigma[n])));
        if (ac[n,c,t]>-1) ac[n,c,t]~bernoulli_logit(mu[n,c][2]);
    }}}   
}'''

def estimateOneGroup(data):
    sm=pystan.StanModel(model_code=modelOneGroup)
    dat={'N':data.shape[0],'T':data.shape[1],'rt':data[:,:,0],'ac':np.int32(data[:,:,1])}
    fit=smBoth.sampling(data=dat,iter=2000,chains=6,seed=np.random.randint(2**16),
                        warmup=1000,thin=1,n_jobs=6)
    saveStanFit(fit,'oneGroup')
    w=fit.extract()
    return (w['mu'],w['hm'],w['hsd'],w['r'][:,0,1],w['sigma'])

def estimateMultiCondSameCov(*args,**kwargs):
    '''mutiple conditions with identical covariance matrix and hence identical sd and r'''
    C=len(args)
    inputPars = kwargs.get('inputPars',"random")
    T=max(map(lambda x: x.shape[1],args))
    N=args[0].shape[0]
    data=np.zeros((N,C,T,2))
    maxT=[]
    for c in range(C):
        maxT.append(args[c].shape[1])
        data[:,c,:maxT[c],:]=args[c]
    print 'fitting model'
    sm=pystan.StanModel(model_code=modelMultiCondSameTheta)
    dat={'N':N,'T':T,'C':C,'rt':data[:,:,:,0],'ac':np.int32(data[:,:,:,1]),'maxT':maxT}
    fit=sm.sampling(data=dat,iter=2000,chains=6,seed=np.random.randint(2**16),
                        warmup=1000,thin=1,n_jobs=6,init=inputPars)
    saveStanFit(fit,'multiCondSameCov')
    w=fit.extract()
    print 'finished'
    return w['mu'],w['hm'],w['hsd'],w['r'][:,0,1],w['sigma']

In [81]:
ipath='/home/matus/Desktop/chase/behavioralOutput/'

def loadData(vpn, verbose=False):
    D=[]
    for vp in vpn:
        dat=np.loadtxt(ipath+'vp%03d.res'%vp)
        if verbose: print vp, dat.shape
        D.append(dat[dat[:,1]>0,:])
    D=np.concatenate(D,0)
    return D

BLMAX=4 # maximum number of block per subject
T1=36
T2=4
vpn=range(20,70) # subject ids
D=loadData(vpn)

dat1=np.zeros((len(vpn),BLMAX*T1,2))
dat2=np.zeros((len(vpn),BLMAX*T2,2))
dat1[:,:,1]=-1;dat2[:,:,1]=-1
for i in range(len(vpn)):
    sel= D[:,0]==vpn[i]
    acc= D[sel,-1]
    rts= D[sel,6]
    acc[rts==30]=-1
    rts[rts==30]=-30
    cond=D[sel,3]<36
    dat1[i,:cond.sum(),0]=rts[cond]
    dat2[i,:(~cond).sum(),0]=rts[~cond]
    dat1[i,:cond.sum(),1]=acc[cond]
    dat2[i,:(~cond).sum(),1]=acc[~cond]
ipr=6*[{'sigma':5*np.ones(N),'mu':np.ones((N,C,2)),'hm':[[1,1],[1,1]],
            'hsd':[1,2],'r':[[1,0.5],[0.5,1]]}]
pts,mu,sd,r,sigma=estimateMultiCondSameCov(dat1,dat2,inputPars=ipr)

fitting model
finished


/usr/lib/python2.7/multiprocessing/queues.py:392: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  return send(obj)
/usr/lib/python2.7/multiprocessing/queues.py:392: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  return send(obj)
/usr/lib/python2.7/multiprocessing/queues.py:392: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  return send(obj)
/usr/lib/python2.7/multiprocessing/queues.py:392: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanMo